In [1]:
import numpy as np
import pandas as pd
import copy

Loading the dictionary from 'Gathering Data' file

In [2]:
list_of_dicts = np.load('list_of_dicts.npy',allow_pickle=True)

In [3]:
pd.DataFrame([list_of_dicts[0][2]])

,address,propertyType,bedrooms,images,hasFloorPlan,transactions,location,detailUrl
0,"118, Newport Road, London, Greater London E10 6PG",Flat,2,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"[{'displayPrice': '£504,000', 'dateSold': '29 ...","{'lat': 51.56547, 'lng': -0.00145}",https://www.rightmove.co.uk/house-prices/detai...


## Data Manipulation

Creating a dataframe based on the loaded dictionary

In [4]:
list_of_df = []
for dict in list_of_dicts:
    for i in dict:
        list_of_df.append(i)

prop_df = pd.DataFrame(list_of_df)
prop_df.head()

,address,propertyType,bedrooms,images,hasFloorPlan,transactions,location,detailUrl
0,"11, Whitney Road, London, Greater London E10 7HG",Terraced,3.0,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"[{'displayPrice': '£720,050', 'dateSold': '17 ...","{'lat': 51.57167, 'lng': -0.01677}",https://www.rightmove.co.uk/house-prices/detai...
1,"82, Manor Road, Leyton, London, Greater London...",Terraced,4.0,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"[{'displayPrice': '£643,756', 'dateSold': '9 D...","{'lat': 51.57082, 'lng': -0.01955}",https://www.rightmove.co.uk/house-prices/detai...
2,"118, Newport Road, London, Greater London E10 6PG",Flat,2.0,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"[{'displayPrice': '£504,000', 'dateSold': '29 ...","{'lat': 51.56547, 'lng': -0.00145}",https://www.rightmove.co.uk/house-prices/detai...
3,"8, Holden House, 616, High Road Leyton, London...",Flat,NaN,{'imageUrl': '/spw/images/placeholder/no-image...,False,"[{'displayPrice': '£130,000', 'dateSold': '26 ...","{'lat': 51.56901, 'lng': -0.00823}",
4,"3, Buckingham Road, London, Greater London E10...",Terraced,3.0,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"[{'displayPrice': '£714,000', 'dateSold': '18 ...","{'lat': 51.56015, 'lng': -0.01063}",https://www.rightmove.co.uk/house-prices/detai...


As you can see there is a need to further unpack some columns before I can conduct Exploratory Data Analysis. For example, 'transactions' needs to be further broken down to extract more features. Furthermore, 'transactions' can take lists and therefore takes multiple values for the same address.

In [5]:
prop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 975 entries, 0 to 974
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   address       975 non-null    object 
 1   propertyType  975 non-null    object 
 2   bedrooms      586 non-null    float64
 3   images        975 non-null    object 
 4   hasFloorPlan  975 non-null    bool   
 5   transactions  975 non-null    object 
 6   location      975 non-null    object 
 7   detailUrl     975 non-null    object 
dtypes: bool(1), float64(1), object(6)
memory usage: 54.4+ KB


Furthermore, according to the website there should be 7,917 results compared to 975 here. I suspect this is because an indiviudal address can be further split through the 'transactions' column.

I will first manipulate one entry within the list. There is a need to unpack 'location', 'images' and 'transactions'.

In [6]:
df0 = pd.DataFrame([dict[0]])
df0

,address,propertyType,bedrooms,images,hasFloorPlan,transactions,location,detailUrl
0,"4, Manor Road, Leyton, London, Greater London ...",Semi-Detached,5,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"[{'displayPrice': '£830,000', 'dateSold': '23 ...","{'lat': 51.56845, 'lng': -0.01792}",https://www.rightmove.co.uk/house-prices/detai...


In [7]:
df0['transactions']

0    [{'displayPrice': '£830,000', 'dateSold': '23 ...
Name: transactions, dtype: object

In [8]:
df0['transactions'][0]

[{'displayPrice': '£830,000',
  'dateSold': '23 Jul 2019',
  'tenure': 'Freehold',
  'newBuild': False},
 {'displayPrice': '£103,500',
  'dateSold': '30 Jan 1995',
  'tenure': 'Freehold',
  'newBuild': False}]

I need to create a dataframe for each transaction value.

In [9]:
lst = []

for i in df0['transactions']:
    x = pd.DataFrame(i)
    lst.append(x)

pd.concat(lst)

,displayPrice,dateSold,tenure,newBuild
0,"£830,000",23 Jul 2019,Freehold,False
1,"£103,500",30 Jan 1995,Freehold,False


I learnt that its better to form a list first then form a dataframe rather than producing a dataframe iterably then appending it

In [10]:
lst = []

for i in df0['transactions']:
    for j in i:
        lst.append(j)

pd.DataFrame(lst)

,displayPrice,dateSold,tenure,newBuild
0,"£830,000",23 Jul 2019,Freehold,False
1,"£103,500",30 Jan 1995,Freehold,False


A method to apply over the entire dataframe

In [11]:
def unpack_tran(x):
    lst = []

    for i in x:
        i['transactions'] = x
        lst.append(i)

    return lst

In [12]:
temp_list = [x for y in prop_df['transactions'].apply(unpack_tran) for x in y]
temp_list[0]

{'displayPrice': '£720,050',
 'dateSold': '17 Dec 2021',
 'tenure': 'Freehold',
 'newBuild': False,
 'transactions': [{...},
  {'displayPrice': '£445,000',
   'dateSold': '12 Aug 2015',
   'tenure': 'Freehold',
   'newBuild': False,
   'transactions': [...]}]}

In [13]:
len(temp_list)

2178

In [14]:
tran_df = pd.DataFrame(temp_list)
tran_df.head()

,displayPrice,dateSold,tenure,newBuild,transactions
0,"£720,050",17 Dec 2021,Freehold,False,"[{'displayPrice': '£720,050', 'dateSold': '17 ..."
1,"£445,000",12 Aug 2015,Freehold,False,"[{'displayPrice': '£720,050', 'dateSold': '17 ..."
2,"£643,756",9 Dec 2021,Freehold,False,"[{'displayPrice': '£643,756', 'dateSold': '9 D..."
3,"£355,000",7 Sep 2007,Freehold,False,"[{'displayPrice': '£643,756', 'dateSold': '9 D..."
4,"£220,000",23 Mar 2006,Freehold,False,"[{'displayPrice': '£643,756', 'dateSold': '9 D..."


In [15]:
key_list1 = [x for x in tran_df['transactions'].apply(lambda x: str(x))]

Created a key using 'transactions' and manipulating it as a key

In [16]:
tran_df['key'] = pd.Series(key_list1)

In [17]:
tran_df.head()

,displayPrice,dateSold,tenure,newBuild,transactions,key
0,"£720,050",17 Dec 2021,Freehold,False,"[{'displayPrice': '£720,050', 'dateSold': '17 ...","[{'displayPrice': '£720,050', 'dateSold': '17 ..."
1,"£445,000",12 Aug 2015,Freehold,False,"[{'displayPrice': '£720,050', 'dateSold': '17 ...","[{'displayPrice': '£720,050', 'dateSold': '17 ..."
2,"£643,756",9 Dec 2021,Freehold,False,"[{'displayPrice': '£643,756', 'dateSold': '9 D...","[{'displayPrice': '£643,756', 'dateSold': '9 D..."
3,"£355,000",7 Sep 2007,Freehold,False,"[{'displayPrice': '£643,756', 'dateSold': '9 D...","[{'displayPrice': '£643,756', 'dateSold': '9 D..."
4,"£220,000",23 Mar 2006,Freehold,False,"[{'displayPrice': '£643,756', 'dateSold': '9 D...","[{'displayPrice': '£643,756', 'dateSold': '9 D..."


Now time to merge the two dataframes together. I will make 'transactions' a key to merge the two dataframes by converting 'transactions' into strings 'key'

In [18]:
key_list = [x for x in prop_df['transactions'].apply(lambda x: str(x))]

I'm also creating a key from the main dataframe using 'transactions' and converting into a string

In [19]:
prop_df['key'] = pd.Series(key_list)
prop_df.head()

,address,propertyType,bedrooms,images,hasFloorPlan,transactions,location,detailUrl,key
0,"11, Whitney Road, London, Greater London E10 7HG",Terraced,3.0,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"[{'displayPrice': '£720,050', 'dateSold': '17 ...","{'lat': 51.57167, 'lng': -0.01677}",https://www.rightmove.co.uk/house-prices/detai...,"[{'displayPrice': '£720,050', 'dateSold': '17 ..."
1,"82, Manor Road, Leyton, London, Greater London...",Terraced,4.0,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"[{'displayPrice': '£643,756', 'dateSold': '9 D...","{'lat': 51.57082, 'lng': -0.01955}",https://www.rightmove.co.uk/house-prices/detai...,"[{'displayPrice': '£643,756', 'dateSold': '9 D..."
2,"118, Newport Road, London, Greater London E10 6PG",Flat,2.0,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"[{'displayPrice': '£504,000', 'dateSold': '29 ...","{'lat': 51.56547, 'lng': -0.00145}",https://www.rightmove.co.uk/house-prices/detai...,"[{'displayPrice': '£504,000', 'dateSold': '29 ..."
3,"8, Holden House, 616, High Road Leyton, London...",Flat,NaN,{'imageUrl': '/spw/images/placeholder/no-image...,False,"[{'displayPrice': '£130,000', 'dateSold': '26 ...","{'lat': 51.56901, 'lng': -0.00823}",,"[{'displayPrice': '£130,000', 'dateSold': '26 ..."
4,"3, Buckingham Road, London, Greater London E10...",Terraced,3.0,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"[{'displayPrice': '£714,000', 'dateSold': '18 ...","{'lat': 51.56015, 'lng': -0.01063}",https://www.rightmove.co.uk/house-prices/detai...,"[{'displayPrice': '£714,000', 'dateSold': '18 ..."


In [21]:
prop_df = pd.merge(tran_df, prop_df, how='inner', on='key')
prop_df.head()

,displayPrice,dateSold,tenure,newBuild,transactions_x,key,address,propertyType,bedrooms,images,hasFloorPlan,transactions_y,location,detailUrl
0,"£720,050",17 Dec 2021,Freehold,False,"[{'displayPrice': '£720,050', 'dateSold': '17 ...","[{'displayPrice': '£720,050', 'dateSold': '17 ...","11, Whitney Road, London, Greater London E10 7HG",Terraced,3.0,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"[{'displayPrice': '£720,050', 'dateSold': '17 ...","{'lat': 51.57167, 'lng': -0.01677}",https://www.rightmove.co.uk/house-prices/detai...
1,"£445,000",12 Aug 2015,Freehold,False,"[{'displayPrice': '£720,050', 'dateSold': '17 ...","[{'displayPrice': '£720,050', 'dateSold': '17 ...","11, Whitney Road, London, Greater London E10 7HG",Terraced,3.0,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"[{'displayPrice': '£720,050', 'dateSold': '17 ...","{'lat': 51.57167, 'lng': -0.01677}",https://www.rightmove.co.uk/house-prices/detai...
2,"£643,756",9 Dec 2021,Freehold,False,"[{'displayPrice': '£643,756', 'dateSold': '9 D...","[{'displayPrice': '£643,756', 'dateSold': '9 D...","82, Manor Road, Leyton, London, Greater London...",Terraced,4.0,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"[{'displayPrice': '£643,756', 'dateSold': '9 D...","{'lat': 51.57082, 'lng': -0.01955}",https://www.rightmove.co.uk/house-prices/detai...
3,"£355,000",7 Sep 2007,Freehold,False,"[{'displayPrice': '£643,756', 'dateSold': '9 D...","[{'displayPrice': '£643,756', 'dateSold': '9 D...","82, Manor Road, Leyton, London, Greater London...",Terraced,4.0,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"[{'displayPrice': '£643,756', 'dateSold': '9 D...","{'lat': 51.57082, 'lng': -0.01955}",https://www.rightmove.co.uk/house-prices/detai...
4,"£220,000",23 Mar 2006,Freehold,False,"[{'displayPrice': '£643,756', 'dateSold': '9 D...","[{'displayPrice': '£643,756', 'dateSold': '9 D...","82, Manor Road, Leyton, London, Greater London...",Terraced,4.0,{'imageUrl': 'https://media.rightmove.co.uk/di...,True,"[{'displayPrice': '£643,756', 'dateSold': '9 D...","{'lat': 51.57082, 'lng': -0.01955}",https://www.rightmove.co.uk/house-prices/detai...


In [22]:
prop_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2278 entries, 0 to 2277
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   displayPrice    2278 non-null   object 
 1   dateSold        2278 non-null   object 
 2   tenure          2278 non-null   object 
 3   newBuild        2278 non-null   bool   
 4   transactions_x  2278 non-null   object 
 5   key             2278 non-null   object 
 6   address         2278 non-null   object 
 7   propertyType    2278 non-null   object 
 8   bedrooms        1588 non-null   float64
 9   images          2278 non-null   object 
 10  hasFloorPlan    2278 non-null   bool   
 11  transactions_y  2278 non-null   object 
 12  location        2278 non-null   object 
 13  detailUrl       2278 non-null   object 
dtypes: bool(2), float64(1), object(11)
memory usage: 235.8+ KB


I will now unpack 'images' and 'location' for the one entry

In [23]:
df0['images'][0]['imageUrl']

'https://media.rightmove.co.uk/dir/33k/32706/66821818/32706_S41300_IMG_07_0000_max_135x100.jpg'

In [24]:
df0['images'] = df0['images'][0]['imageUrl']
df0

,address,propertyType,bedrooms,images,hasFloorPlan,transactions,location,detailUrl
0,"4, Manor Road, Leyton, London, Greater London ...",Semi-Detached,5,https://media.rightmove.co.uk/dir/33k/32706/66...,True,"[{'displayPrice': '£830,000', 'dateSold': '23 ...","{'lat': 51.56845, 'lng': -0.01792}",https://www.rightmove.co.uk/house-prices/detai...


In [25]:
def unpack_image(x):
    return x['imageUrl']

In [26]:
df0['latitude'] = df0['location'][0]['lat']
df0['longitude'] = df0['location'][0]['lng']
df0


,address,propertyType,bedrooms,images,hasFloorPlan,transactions,location,detailUrl,latitude,longitude
0,"4, Manor Road, Leyton, London, Greater London ...",Semi-Detached,5,https://media.rightmove.co.uk/dir/33k/32706/66...,True,"[{'displayPrice': '£830,000', 'dateSold': '23 ...","{'lat': 51.56845, 'lng': -0.01792}",https://www.rightmove.co.uk/house-prices/detai...,51.56845,-0.01792


In [27]:
df0[['a','b']] = [df0['location'][0]['lat'],df0['location'][0]['lng']]
df0

,address,propertyType,bedrooms,images,hasFloorPlan,transactions,location,detailUrl,latitude,longitude,a,b
0,"4, Manor Road, Leyton, London, Greater London ...",Semi-Detached,5,https://media.rightmove.co.uk/dir/33k/32706/66...,True,"[{'displayPrice': '£830,000', 'dateSold': '23 ...","{'lat': 51.56845, 'lng': -0.01792}",https://www.rightmove.co.uk/house-prices/detai...,51.56845,-0.01792,51.56845,-0.01792


In [28]:
df0[['latitude','longitude']] = [1,1]
df0

,address,propertyType,bedrooms,images,hasFloorPlan,transactions,location,detailUrl,latitude,longitude,a,b
0,"4, Manor Road, Leyton, London, Greater London ...",Semi-Detached,5,https://media.rightmove.co.uk/dir/33k/32706/66...,True,"[{'displayPrice': '£830,000', 'dateSold': '23 ...","{'lat': 51.56845, 'lng': -0.01792}",https://www.rightmove.co.uk/house-prices/detai...,1,1,51.56845,-0.01792


In [29]:
def unpack_loc(x):
    return pd.Series([x['lat'],x['lng']])

In [30]:
df0[['latitude','longitude']] = df0['location'].apply(unpack_loc)
df0

,address,propertyType,bedrooms,images,hasFloorPlan,transactions,location,detailUrl,latitude,longitude,a,b
0,"4, Manor Road, Leyton, London, Greater London ...",Semi-Detached,5,https://media.rightmove.co.uk/dir/33k/32706/66...,True,"[{'displayPrice': '£830,000', 'dateSold': '23 ...","{'lat': 51.56845, 'lng': -0.01792}",https://www.rightmove.co.uk/house-prices/detai...,51.56845,-0.01792,51.56845,-0.01792


Time to unpack for the rest of the dictionaries.

In [31]:
prop_df['images'] = prop_df['images'].apply(unpack_image)

prop_df.head()

,displayPrice,dateSold,tenure,newBuild,transactions_x,key,address,propertyType,bedrooms,images,hasFloorPlan,transactions_y,location,detailUrl
0,"£720,050",17 Dec 2021,Freehold,False,"[{'displayPrice': '£720,050', 'dateSold': '17 ...","[{'displayPrice': '£720,050', 'dateSold': '17 ...","11, Whitney Road, London, Greater London E10 7HG",Terraced,3.0,https://media.rightmove.co.uk/dir/66k/65394/10...,True,"[{'displayPrice': '£720,050', 'dateSold': '17 ...","{'lat': 51.57167, 'lng': -0.01677}",https://www.rightmove.co.uk/house-prices/detai...
1,"£445,000",12 Aug 2015,Freehold,False,"[{'displayPrice': '£720,050', 'dateSold': '17 ...","[{'displayPrice': '£720,050', 'dateSold': '17 ...","11, Whitney Road, London, Greater London E10 7HG",Terraced,3.0,https://media.rightmove.co.uk/dir/66k/65394/10...,True,"[{'displayPrice': '£720,050', 'dateSold': '17 ...","{'lat': 51.57167, 'lng': -0.01677}",https://www.rightmove.co.uk/house-prices/detai...
2,"£643,756",9 Dec 2021,Freehold,False,"[{'displayPrice': '£643,756', 'dateSold': '9 D...","[{'displayPrice': '£643,756', 'dateSold': '9 D...","82, Manor Road, Leyton, London, Greater London...",Terraced,4.0,https://media.rightmove.co.uk/dir/71k/70202/11...,True,"[{'displayPrice': '£643,756', 'dateSold': '9 D...","{'lat': 51.57082, 'lng': -0.01955}",https://www.rightmove.co.uk/house-prices/detai...
3,"£355,000",7 Sep 2007,Freehold,False,"[{'displayPrice': '£643,756', 'dateSold': '9 D...","[{'displayPrice': '£643,756', 'dateSold': '9 D...","82, Manor Road, Leyton, London, Greater London...",Terraced,4.0,https://media.rightmove.co.uk/dir/71k/70202/11...,True,"[{'displayPrice': '£643,756', 'dateSold': '9 D...","{'lat': 51.57082, 'lng': -0.01955}",https://www.rightmove.co.uk/house-prices/detai...
4,"£220,000",23 Mar 2006,Freehold,False,"[{'displayPrice': '£643,756', 'dateSold': '9 D...","[{'displayPrice': '£643,756', 'dateSold': '9 D...","82, Manor Road, Leyton, London, Greater London...",Terraced,4.0,https://media.rightmove.co.uk/dir/71k/70202/11...,True,"[{'displayPrice': '£643,756', 'dateSold': '9 D...","{'lat': 51.57082, 'lng': -0.01955}",https://www.rightmove.co.uk/house-prices/detai...


In [32]:
prop_df[['latitude','longitude']] = prop_df['location'].apply(unpack_loc)
prop_df.head()

,displayPrice,dateSold,tenure,newBuild,transactions_x,key,address,propertyType,bedrooms,images,hasFloorPlan,transactions_y,location,detailUrl,latitude,longitude
0,"£720,050",17 Dec 2021,Freehold,False,"[{'displayPrice': '£720,050', 'dateSold': '17 ...","[{'displayPrice': '£720,050', 'dateSold': '17 ...","11, Whitney Road, London, Greater London E10 7HG",Terraced,3.0,https://media.rightmove.co.uk/dir/66k/65394/10...,True,"[{'displayPrice': '£720,050', 'dateSold': '17 ...","{'lat': 51.57167, 'lng': -0.01677}",https://www.rightmove.co.uk/house-prices/detai...,51.57167,-0.01677
1,"£445,000",12 Aug 2015,Freehold,False,"[{'displayPrice': '£720,050', 'dateSold': '17 ...","[{'displayPrice': '£720,050', 'dateSold': '17 ...","11, Whitney Road, London, Greater London E10 7HG",Terraced,3.0,https://media.rightmove.co.uk/dir/66k/65394/10...,True,"[{'displayPrice': '£720,050', 'dateSold': '17 ...","{'lat': 51.57167, 'lng': -0.01677}",https://www.rightmove.co.uk/house-prices/detai...,51.57167,-0.01677
2,"£643,756",9 Dec 2021,Freehold,False,"[{'displayPrice': '£643,756', 'dateSold': '9 D...","[{'displayPrice': '£643,756', 'dateSold': '9 D...","82, Manor Road, Leyton, London, Greater London...",Terraced,4.0,https://media.rightmove.co.uk/dir/71k/70202/11...,True,"[{'displayPrice': '£643,756', 'dateSold': '9 D...","{'lat': 51.57082, 'lng': -0.01955}",https://www.rightmove.co.uk/house-prices/detai...,51.57082,-0.01955
3,"£355,000",7 Sep 2007,Freehold,False,"[{'displayPrice': '£643,756', 'dateSold': '9 D...","[{'displayPrice': '£643,756', 'dateSold': '9 D...","82, Manor Road, Leyton, London, Greater London...",Terraced,4.0,https://media.rightmove.co.uk/dir/71k/70202/11...,True,"[{'displayPrice': '£643,756', 'dateSold': '9 D...","{'lat': 51.57082, 'lng': -0.01955}",https://www.rightmove.co.uk/house-prices/detai...,51.57082,-0.01955
4,"£220,000",23 Mar 2006,Freehold,False,"[{'displayPrice': '£643,756', 'dateSold': '9 D...","[{'displayPrice': '£643,756', 'dateSold': '9 D...","82, Manor Road, Leyton, London, Greater London...",Terraced,4.0,https://media.rightmove.co.uk/dir/71k/70202/11...,True,"[{'displayPrice': '£643,756', 'dateSold': '9 D...","{'lat': 51.57082, 'lng': -0.01955}",https://www.rightmove.co.uk/house-prices/detai...,51.57082,-0.01955


Cleaning the dataframe from duplicate and unpacked columns

In [34]:
prop_df.drop(['transactions_x','transactions_y','location','key'],axis=1,inplace=True)

In [35]:
prop_df.head()

,displayPrice,dateSold,tenure,newBuild,address,propertyType,bedrooms,images,hasFloorPlan,detailUrl,latitude,longitude
0,"£720,050",17 Dec 2021,Freehold,False,"11, Whitney Road, London, Greater London E10 7HG",Terraced,3.0,https://media.rightmove.co.uk/dir/66k/65394/10...,True,https://www.rightmove.co.uk/house-prices/detai...,51.57167,-0.01677
1,"£445,000",12 Aug 2015,Freehold,False,"11, Whitney Road, London, Greater London E10 7HG",Terraced,3.0,https://media.rightmove.co.uk/dir/66k/65394/10...,True,https://www.rightmove.co.uk/house-prices/detai...,51.57167,-0.01677
2,"£643,756",9 Dec 2021,Freehold,False,"82, Manor Road, Leyton, London, Greater London...",Terraced,4.0,https://media.rightmove.co.uk/dir/71k/70202/11...,True,https://www.rightmove.co.uk/house-prices/detai...,51.57082,-0.01955
3,"£355,000",7 Sep 2007,Freehold,False,"82, Manor Road, Leyton, London, Greater London...",Terraced,4.0,https://media.rightmove.co.uk/dir/71k/70202/11...,True,https://www.rightmove.co.uk/house-prices/detai...,51.57082,-0.01955
4,"£220,000",23 Mar 2006,Freehold,False,"82, Manor Road, Leyton, London, Greater London...",Terraced,4.0,https://media.rightmove.co.uk/dir/71k/70202/11...,True,https://www.rightmove.co.uk/house-prices/detai...,51.57082,-0.01955


Saving and exporting the manipulated dataframe for Exploratory Data Analysis

In [36]:
prop_df.to_csv('Property df.csv')